# Programa auxiliar para separar las Imágenes, XMLs y/o LABELs para Entrenamiento, Validación y/o Prueba

In [1]:
#@title Cargar Librerías
import os
import os.path
import numpy as np
import shutil

import zipfile

print ("Librerías cargadas.")

Librerías cargadas.


In [2]:
#@title Montar Google Drive
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [3]:
#@title Definir parámetros a utilizar

# directorio local en Google Drive
#@markdown Carpeta principal:
path = '/content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles' #@param {type:"string"}
#@markdown Carpeta Origen:
subdir_Origen = '/Generados' #@param {type:"string"}

#@markdown Carpetas Destino:
subdir_Entrenamiento = '/train' #@param {type:"string"}
subdir_Validacion = '/validation' #@param {type:"string"}
subdir_Prueba = '/test' #@param {type:"string"}

#@markdown SubCarpetas tipo de Datos:
subdir_subdir_Imagenes = '/images'  #@param {type:"string"}
subdir_subdir_XML = '/annotations'  #@param {type:"string"}
subdir_subdir_LABEL = '/labels'  #@param {type:"string"}

#@markdown Tipos de Archivos:
ext_archivo_Imagen = ".jpg, .jpeg, .png" #@param {type:"string"}
ext_archivo_XML = ".xml" #@param {type:"string"}
ext_archivo_LABEL = ".txt" #@param {type:"string"}

# prepara extensiones
image_extensions = []
for e in ext_archivo_Imagen.split(","):
  e = e.strip()
  if "." not in e:
    e = e + "."
  image_extensions.append(e)

if "." not in ext_archivo_XML:
  ext_archivo_XML = "." + ext_archivo_XML

if "." not in ext_archivo_LABEL:
  ext_archivo_LABEL = "." + ext_archivo_LABEL

# determina archivos a mover
 ## siempre mueve imágenes!
 ##( subdir_subdir_Imagenes!="" and ext_archivo_Imagen!="" and len(image_extensions)>0)
mueveImagenes = True
mueveXML = ( subdir_subdir_XML!="" and ext_archivo_XML!="" )
mueveLABEL = ( subdir_subdir_LABEL!="" and ext_archivo_LABEL!="" )

# determina directorios
dirOrigen = path + subdir_Origen

if subdir_Entrenamiento == "":
  mueveTrain = False
else:
  dirDestTrain = path + subdir_Entrenamiento
  mueveTrain = True

if subdir_Validacion == "":
  mueveVal = False
else:
  dirDestVal = path + subdir_Validacion
  mueveVal = True

if subdir_Prueba == "":
  dirDestTest = None
  mueveTest = False
else:
  dirDestTest = path + subdir_Prueba
  mueveTest = True

print("\n -- Mueve ")
print("   [ \n\t",
    ("Imágenes" if mueveImagenes else ""),
    ("XMLs" if mueveXML else ""),
    ("LABELs" if mueveLABEL else ""),
       "\n   ] ")
print(" --> ")
print("   [",
    ("\t"+dirDestTrain+"\n" if mueveTrain else ""),
    ("\t"+dirDestVal+"\n" if mueveVal else ""),
    ("\t"+dirDestTest+"\n" if mueveTest else ""),
       "   ]" )

print("\n")


 -- Mueve 
   [ 
	 Imágenes XMLs LABELs 
   ] 
 --> 
   [ 	/content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/train
 	/content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/validation
 	/content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/test
    ]




In [4]:
#@title Descomprimir ZIP (OPCIONAL)

# define parámetros
zipFile =  '' #@param {type:"string"}

if zipFile != "":

  destinoFiles = dirOrigen

  # realiza unzip
  zip_ref = zipfile.ZipFile(zipFile, 'r')
  zip_ref.extractall(destinoFiles)
  zip_ref.close()

  # fuerza la actualizacion del drive
  drive.flush_and_unmount()


In [5]:
#@title Chequear archivos disponibles
# vuelve a montar
drive.mount('/content/gdrive', force_remount=True)

def fmtContarArchios(dir, extList=None):
  res = contarArchivos(dir, extList)
  if res < 0:
    return "NO EXISTE!"
  else:
    return str(res)

def contarArchivos(dir, extList=None):
  if not os.path.isdir(dir):
    return -1
  all = os.listdir(dir)
  if (extList is None) or (len(extList)==0):
    return len(all)
  else:
    count = 0
    for fn in all:
      _, ext = os.path.splitext(fn)
      if ext.lower() in extList:
        count = count + 1
    return count

# indica cantidad de archivos extraidos
print("> Todos los Archivos disponibles: ", fmtContarArchios(dirOrigen))
if mueveImagenes:
  print("\t Archivos IMAGEN disponibles: ",  fmtContarArchios(dirOrigen, image_extensions))
if mueveXML:
  print("\t Archivos XML disponibles: ", fmtContarArchios(dirOrigen, [ext_archivo_XML]) )
if mueveLABEL:
  print("\t Archivos LABEL disponibles: ", fmtContarArchios(dirOrigen, [ext_archivo_LABEL]) )

print("\n")

Mounted at /content/gdrive
> Todos los Archivos disponibles:  3000
	 Archivos IMAGEN disponibles:  1000
	 Archivos XML disponibles:  1000
	 Archivos LABEL disponibles:  1000




In [6]:
#@title Determinar archivos a distribuir
porc_Entrenamiento_distribucion_archivos = 85 #@param {type:"slider", min:0, max:100, step:1}
porc_Prueba_distribucion_archivos = 10 #@param {type:"slider", min:0, max:100, step:1}
#@markdown porc_Validacion_distribucion_archivos = 100 - porc_Entrenamiento_distribucion_archivos - porc_Prueba_distribucion_archivos

# valida porcentajes
porc_Entrenamiento_distribucion_archivos = min(max(porc_Entrenamiento_distribucion_archivos, 0), 100)
porc_Prueba_distribucion_archivos = min(max(porc_Prueba_distribucion_archivos, 0), 50)
if porc_Entrenamiento_distribucion_archivos + porc_Prueba_distribucion_archivos > 100:
  porc_Prueba_distribucion_archivos = 100 - porc_Entrenamiento_distribucion_archivos


# crea los directorios, si es necesario
if mueveTrain:
  if not os.path.isdir(dirDestTrain):
    os.makedirs(dirDestTrain)
else:
  porc_Entrenamiento_distribucion_archivos = 0

if mueveVal:
  if not os.path.isdir(dirDestVal):
    os.makedirs(dirDestVal)

if mueveTest:
  if not os.path.isdir(dirDestTest):
    os.makedirs(dirDestTest)
else:
  porc_Prueba_distribucion_archivos = 0

# variables auxiliares
all_images_array = []
allFileNames = []
train_FileNames = []
test_FileNames = []

# controla si el archivo es una imagen
def esFNImagen(fn):
  _, ext = os.path.splitext(fn)
  return ext.lower() in image_extensions

# levanta la lista de imágenes de cada letra
allFileNames = [ fn for fn in os.listdir( dirOrigen ) if esFNImagen(fn) ]
cantImagenes = len(allFileNames)

cant_Entrenamiento = int(cantImagenes * (porc_Entrenamiento_distribucion_archivos/100.))
cant_Prueba = int(cantImagenes * (porc_Prueba_distribucion_archivos/100.))
cant_Validacion = cantImagenes - cant_Entrenamiento - cant_Prueba

# desordena
np.random.shuffle(allFileNames)

# separa datos para Train y para Prueba y para Val
test_FileNames, train_FileNames = np.split(np.array(allFileNames), [cant_Prueba])
if cant_Validacion > 0:
  val_FileNames, train_FileNames = np.split(np.array(train_FileNames), [cant_Validacion])
else:
  val_FileNames = []

print('Total Imágenes Disponibles: ', len(allFileNames))
print('- Imágenes para Entrenamiento: ', len(train_FileNames), "(", round(100*len(train_FileNames)/len(allFileNames), 2), "%)")
print('- Imágenes para Validación: ', len(val_FileNames), "(", round(100*len(val_FileNames)/len(allFileNames), 2), "%)")
print('- Imágenes para Prueba: ', len(test_FileNames), "(", round(100*len(test_FileNames)/len(allFileNames), 2), "%)")
print("\n")

if mueveTrain and len(train_FileNames)==0:
  print("-- no hay imágenes para mover a Entrenamiento!")
  mueveTrain = False

if mueveVal and len(val_FileNames)==0:
  print("-- no hay imágenes para mover a Validación!")
  mueveVal = False

if mueveTest and len(test_FileNames)==0:
  print("-- no hay imágenes para mover a Prueba!")
  mueveTest = False

print("\n")

Total Imágenes Disponibles:  1000
- Imágenes para Entrenamiento:  850 ( 85.0 %)
- Imágenes para Validación:  50 ( 5.0 %)
- Imágenes para Prueba:  100 ( 10.0 %)






In [7]:
#@title Mover archivos

# función auxiliar para mover archivos
def moverArchivos(filesNames, dirOrigen, dirDestino, subdir_Imagenes="/", subdir_XML="/", subdir_LABEL="/", ext_archivo_XML=None, ext_archivo_LABEL=None):
    # inicializa contadores
    cant = 0
    cantXML = 0
    cantLABEL = 0

    # crea destinos
    auxImgDir = dirDestino + subdir_Imagenes
    auxXMLDir = dirDestino + subdir_XML
    auxLABELDir = dirDestino + subdir_LABEL

    if mueveImagenes:
      if not os.path.isdir(auxImgDir):
        os.makedirs(auxImgDir)

    if mueveXML:
      if not os.path.isdir(auxXMLDir):
        os.makedirs(auxXMLDir)

      if ext_archivo_XML is None:
        ext_archivo_XML=".xml"

    if mueveLABEL:
      if not os.path.isdir(auxLABELDir):
        os.makedirs(auxLABELDir)

      if ext_archivo_LABEL is None:
        ext_archivo_LABEL=".txt"

    for nameImage in filesNames:
      name, _ = os.path.splitext(nameImage)
      if mueveXML:
        nameXML = name + ext_archivo_XML
        existeXML = os.path.isfile(dirOrigen + "/" + nameXML)
      else:
        existeXML = True
      if mueveLABEL:
        nameLabel = name + ext_archivo_LABEL
        existeLabel = os.path.isfile(dirOrigen + "/" + nameLabel)
      else:
        existeLabel = True
      # si existe el XML o LABEL
      if existeXML or existeLabel:
        # controla si puede mover la imagen
        if os.path.isfile(auxImgDir + "/" + nameImage):
           print("**", nameImage, " ya existe en ", auxImgDir)
        else:
          shutil.move(dirOrigen + "/" + nameImage, auxImgDir)
          cant = cant + 1
        # mueve el xml
        if mueveXML and existeXML:
          if os.path.isfile(auxXMLDir + '/' + nameXML):
            print("**", nameXML, " ya existe en ", auxXMLDir)
          else:
            shutil.move(dirOrigen + "/" + nameXML, auxXMLDir)
            cantXML = cantXML + 1
        # mueve el label
        if mueveLABEL and existeLabel:
          if os.path.isfile(auxLABELDir + '/' + nameLabel):
            print("**", nameLabel, " ya existe en ", auxLABELDir)
          else:
            shutil.move(dirOrigen + "/" + nameLabel, auxLABELDir)
            cantLABEL = cantLABEL + 1
      else:
        print("**", nameXML, " o ", nameLabel, "NO existen en ", dirOrigen, " - se descarta imagen y XML!")
    print(cant, " imágenes movidas a ", auxImgDir)
    print(cantXML, " XML movidos a ", auxXMLDir)
    print(cantLABEL, " LABELS movidos a ", auxLABELDir)
    print("\n")

if mueveTrain:
  print("\n-Procesando archivos para Entrenamiento...")
  moverArchivos(train_FileNames, dirOrigen, dirDestTrain, subdir_subdir_Imagenes, subdir_subdir_XML, subdir_subdir_LABEL, ext_archivo_XML, ext_archivo_LABEL)

if mueveVal:
  print("\n-Procesando archivos para Validación...")
  moverArchivos(val_FileNames, dirOrigen, dirDestVal, subdir_subdir_Imagenes, subdir_subdir_XML,  subdir_subdir_LABEL, ext_archivo_XML, ext_archivo_LABEL)

if mueveTest:
  print("\n-Procesando archivos para Prueba...")
  moverArchivos(test_FileNames, dirOrigen, dirDestTest, subdir_subdir_Imagenes, subdir_subdir_XML,  subdir_subdir_LABEL, ext_archivo_XML, ext_archivo_LABEL)

print("\n")


-Procesando archivos para Entrenamiento...
850  imágenes movidas a  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/train/images
850  XML movidos a  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/train/annotations
850  LABELS movidos a  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/train/labels



-Procesando archivos para Validación...
50  imágenes movidas a  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/validation/images
50  XML movidos a  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/validation/annotations
50  LABELS movidos a  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/validation/labels



-Procesando archivos para Prueba...
100  imágenes movidas a  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/test/images
100  XML movidos a  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/test/annotations
100  LABELS movidos a  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/C

In [8]:
#@title Fuerza la actualizacion del Drive

# actualiza y desmonta
drive.flush_and_unmount()

# vuelve a montar
drive.mount('/content/gdrive', force_remount=True)

print("Actualización terminada.")
print("\n")

Mounted at /content/gdrive
Actualización terminada.




4) Chequea que se haya pasado todo:

In [9]:
#@title Controlar archivos huérfanos

# función auxiliar
def revisarArchivosHuerfanos(dir1, dir2, ext2):
  cantOK = 0
  auxiError = []
  print("\n> Resultados de controlar ", dir1, " contra ", dir2, ": ")
  if not os.path.isdir(dir1):
    print("**", dir1, " no existe!")
    return
  if not os.path.isdir(dir2):
    print("**", dir2, " no existe!")
    return
  for fn1 in os.listdir( dir1 ):
    name, _ = os.path.splitext(fn1)
    fn2 = name + ext2
    if os.path.isfile(dir2 + '/' + fn2):
      cantOK = cantOK + 1
    else:
      auxiError = fn2
  print(" --- ", cantOK, " archivos OK.")
  print(" --- ", len(auxiError), " archivos Huérfanos movidos: ", auxiError)
  return

if mueveTrain:
  # controla archivos de Entrenamiento
  if mueveXML:
    revisarArchivosHuerfanos(dirDestTrain + subdir_subdir_Imagenes, dirDestTrain + subdir_subdir_XML, ext_archivo_XML)
  if mueveLABEL:
    revisarArchivosHuerfanos(dirDestTrain + subdir_subdir_Imagenes, dirDestTrain + subdir_subdir_LABEL, ext_archivo_LABEL)
  if mueveXML and mueveLABEL:
    revisarArchivosHuerfanos(dirDestTrain + subdir_subdir_XML, dirDestTrain + subdir_subdir_LABEL, ext_archivo_LABEL)
    revisarArchivosHuerfanos(dirDestTrain + subdir_subdir_LABEL, dirDestTrain + subdir_subdir_XML, ext_archivo_XML)

if mueveVal:
  # controla archivos de Validación
  if mueveXML:
    revisarArchivosHuerfanos(dirDestVal + subdir_subdir_Imagenes, dirDestVal + subdir_subdir_XML, ext_archivo_XML)
  if mueveLABEL:
    revisarArchivosHuerfanos(dirDestVal + subdir_subdir_Imagenes, dirDestVal + subdir_subdir_LABEL, ext_archivo_LABEL)
  if mueveXML and mueveLABEL:
    revisarArchivosHuerfanos(dirDestVal + subdir_subdir_XML, dirDestVal + subdir_subdir_LABEL, ext_archivo_LABEL)
    revisarArchivosHuerfanos(dirDestVal + subdir_subdir_LABEL, dirDestVal + subdir_subdir_XML, ext_archivo_XML)

if mueveTest:
  # controla archivos de Prueba
  if mueveXML:
    revisarArchivosHuerfanos(dirDestTest + subdir_subdir_Imagenes, dirDestTest + subdir_subdir_XML, ext_archivo_XML)
  if mueveLABEL:
    revisarArchivosHuerfanos(dirDestTest + subdir_subdir_Imagenes, dirDestTest + subdir_subdir_LABEL, ext_archivo_LABEL)
  if mueveXML and mueveLABEL:
    revisarArchivosHuerfanos(dirDestTest + subdir_subdir_XML, dirDestTest + subdir_subdir_LABEL, ext_archivo_LABEL)
    revisarArchivosHuerfanos(dirDestTest + subdir_subdir_LABEL, dirDestTest + subdir_subdir_XML, ext_archivo_XML)

print("\n")


> Resultados de controlar  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/train/images  contra  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/train/annotations : 
 ---  850  archivos OK.
 ---  0  archivos Huérfanos movidos:  []

> Resultados de controlar  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/train/images  contra  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/train/labels : 
 ---  850  archivos OK.
 ---  0  archivos Huérfanos movidos:  []

> Resultados de controlar  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/train/annotations  contra  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/train/labels : 
 ---  850  archivos OK.
 ---  0  archivos Huérfanos movidos:  []

> Resultados de controlar  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/train/labels  contra  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/train/annotations : 
 ---  850  archivos OK.
 ---  0  archi

In [10]:
#@title Mostrar Cantidades por Tipo Archivo

def contarTipoArchivos(tipoArchivo, subDir, tipoExtList, dirOrigen, dirDestTrain, dirDestVal, dirDestTest):
  # mostrar
  print('\nTotal de ' + tipoArchivo + ' en ', dirOrigen,' : ', fmtContarArchios(dirOrigen, tipoExtList))
  if dirDestTrain is not None:
    print('- Total de ' + tipoArchivo + ' en Entrenamiento: ', fmtContarArchios(dirDestTrain + subDir, tipoExtList))
  if dirDestVal is not None:
    print('- Total de ' + tipoArchivo + ' en Validación: ', fmtContarArchios(dirDestVal + subDir, tipoExtList))
  if dirDestTest is not None:
    print('- Total de ' + tipoArchivo + ' en Prueba: ', fmtContarArchios(dirDestTest + subDir, tipoExtList))
  print("")
  return

# determina directorios a procesar
if mueveTrain:
  dirTrain = dirDestTrain
else:
  dirTrain = None

if mueveVal:
  dirVal = dirDestVal
else:
  dirVal = None

if mueveTest:
  dirTest = dirDestTest
else:
  dirTest = None


# procesar tipos de archivos
if mueveImagenes:
  contarTipoArchivos("Imágenes", subdir_subdir_Imagenes, image_extensions, dirOrigen, dirTrain, dirVal, dirTest)

if mueveXML:
  contarTipoArchivos("XML", subdir_subdir_XML, [ext_archivo_XML], dirOrigen, dirTrain, dirVal, dirTest)

if mueveLABEL:
  contarTipoArchivos("LABEL", subdir_subdir_LABEL, [ext_archivo_LABEL], dirOrigen, dirTrain, dirVal, dirTest)

print("\n")



Total de Imágenes en  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/Generados  :  0
- Total de Imágenes en Entrenamiento:  850
- Total de Imágenes en Validación:  50
- Total de Imágenes en Prueba:  100


Total de XML en  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/Generados  :  0
- Total de XML en Entrenamiento:  850
- Total de XML en Validación:  50
- Total de XML en Prueba:  100


Total de LABEL en  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/Generados  :  0
- Total de LABEL en Entrenamiento:  850
- Total de LABEL en Validación:  50
- Total de LABEL en Prueba:  100



